In [1]:
# nuclio: ignore
import nuclio

In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import dask
import dask.dataframe as dd
from dask.distributed import Client

from mlrun.execution import MLClientCtx
from mlrun.datastore import DataItem
from mlrun.artifacts import PlotArtifact, TableArtifact
from mlrun.mlutils import gcf_clear

from yellowbrick import ClassBalance

from typing import List

/User/.pythonlibs/jupyter/lib/python3.6/site-packages/dask/dataframe/utils.py:14: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
/conda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.classification module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)


In [3]:
pd.set_option("display.float_format", lambda x: "%.2f" % x)

def summarize(
    context: MLClientCtx,
    dask_key: str = "dask_key",
    label_column: str = "labels",
    class_labels: List[str] = [],
    plot_hist: bool = True,
    plots_dest: str = "plots",
    alt_scheduler: str = None
) -> None:
    """Summarize a table
    
    Connects to dask client through the function context, or through an optional
    user-supplied scheduler.

    :param context:         the function context
    :param dask_key:        key of dataframe in dask client "datasets" attribute
    :param label_column:    ground truth column label
    :param class_labels:    label for each class in tables and plots
    :param plot_hist:       (True) set this to False for large tables
    :param plots_dest:      destination folder of summary plots (relative to artifact_path)
    :param alt_scheduler:   (None) an alternative scheduler file to connect with
    """
    if alt_scheduler:
        dask_client = Client(scheduler_file=str(alt_scheduler))
    elif hasattr(context, "dask_client"):
        dask_client = Client(scheduler_file=str(context.dask_client))
    else:
        raise Exception("out of luck, no dask_client or scheduler file!")
        
    if dask_key in dask_client.datasets:
        table = dask_client.get_dataset(dask_key)
    else:
        context.logger.info(f"only these datasets are available {dask_client.datasets} in client {dask_client}")
        raise Exception("dataset not found on dask cluster")
    header = table.columns.values
    
    gcf_clear(plt)
    table = table.compute()
    snsplt = sns.pairplot(table, hue=label_column, diag_kws={'bw': 1.5})
    context.log_artifact(PlotArtifact('histograms',  body=plt.gcf()), 
                         local_path=f"{plots_dest}/hist.html")

    gcf_clear(plt)   
    labels = table.pop(label_column)
    if not class_labels:
        class_labels = labels.unique()
    class_balance_model = ClassBalance(labels=class_labels)
    class_balance_model.fit(labels)   
    scale_pos_weight = class_balance_model.support_[0]/class_balance_model.support_[1]
    context.log_result("scale_pos_weight", f"{scale_pos_weight:0.2f}")
    context.log_artifact(PlotArtifact("imbalance", body=plt.gcf()), 
                         local_path=f"{plots_dest}/imbalance.html")
    
    gcf_clear(plt)
    tblcorr = table.corr()
    ax = plt.axes()
    sns.heatmap(tblcorr, ax=ax, annot=False, cmap=plt.cm.Reds)
    ax.set_title("features correlation")
    context.log_artifact(PlotArtifact("correlation",  body=plt.gcf()), 
                         local_path=f"{plots_dest}/corr.html")
    # otherwise shows last plot:
    gcf_clear(plt)

In [4]:
# nuclio: end-code

In [5]:
from mlrun import run_local, code_to_function , NewTask, mlconf

mlconf.dbpath = "http://mlrun-api:8080"
mlconf.artifact_path = '/User/artifacts'

In [6]:
# create job function object from notebook code
fn = code_to_function('describe_dask', kind='job', with_doc=True,
                      handler=summarize, image='mlrun/ml-models')

# add metadata (for templates and reuse)
fn.spec.default_handler = 'summarize'
fn.spec.description = "describe and visualizes dataset stats"
fn.metadata.categories = ['models', 'visualization']
fn.metadata.labels = {'author': 'yjb'}

fn.save()
fn.export('function.yaml')

[mlrun] 2020-03-31 17:28:58,860 saving function: describe-dask, tag: latest
[mlrun] 2020-03-31 17:28:58,901 function spec saved to path: function.yaml


In [7]:
table_path = os.path.join(mlconf.artifact_path, "iris.parquet")
task = NewTask(name="tasks describe dask", 
               handler=summarize, 
               inputs={'dask_key': "dask_key",
                       # the scheduler from load_dask
                       "alt_scheduler" :"/User/artifacts/scheduler.json"})
run = run_local(task)

[mlrun] 2020-03-31 17:28:58,915 starting run tasks describe dask uid=b65804d2fb504d92b20fa01c4c7ec3b4  -> http://mlrun-api:8080


findfont: Font family ['sans-serif'] not found. Falling back to DejaVu Sans.
findfont: Font family ['sans-serif'] not found. Falling back to DejaVu Sans.
findfont: Font family ['sans-serif'] not found. Falling back to DejaVu Sans.
findfont: Font family ['sans-serif'] not found. Falling back to DejaVu Sans.


[mlrun] 2020-03-31 17:32:04,475 log artifact histograms at /User/artifacts/b65804d2fb504d92b20fa01c4c7ec3b4/plots/hist.html, size: 5675657, db: Y
[mlrun] 2020-03-31 17:32:27,926 log artifact imbalance at /User/artifacts/b65804d2fb504d92b20fa01c4c7ec3b4/plots/imbalance.html, size: 10756, db: Y
[mlrun] 2020-03-31 17:32:28,270 log artifact correlation at /User/artifacts/b65804d2fb504d92b20fa01c4c7ec3b4/plots/corr.html, size: 54930, db: Y



uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
...7ec3b4,0,Mar 31 17:28:58,completed,tasks describe dask,v3io_user=adminkind=handlerowner=adminhost=jupyter-57954c4b4c-xln99,dask_keyalt_scheduler,,scale_pos_weight=0.59,histogramsimbalancecorrelation


to track results use .show() or .logs() or in CLI: 
!mlrun get run b65804d2fb504d92b20fa01c4c7ec3b4 --project default , !mlrun logs b65804d2fb504d92b20fa01c4c7ec3b4 --project default
[mlrun] 2020-03-31 17:32:28,389 run executed, status=completed
